# QC & Filtering

In [1]:
!wget -O t2d_bbj.txt.gz http://jenger.riken.jp/14/

--2022-10-31 14:58:57--  http://jenger.riken.jp/14/
Resolving jenger.riken.jp (jenger.riken.jp)... 134.160.84.25
Connecting to jenger.riken.jp (jenger.riken.jp)|134.160.84.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274187574 (261M) [text/plain]
Saving to: ‘t2d_bbj.txt.gz’

t2d_bbj.txt.gz      100%[===================>] 261.49M  37.0MB/s    in 7.1s    

2022-10-31 14:59:05 (36.7 MB/s) - ‘t2d_bbj.txt.gz’ saved [274187574/274187574]



## Loading gwaslab and sumstats

In [1]:
import sys
sys.path.insert(0,"/Users/he/work/gwaslab/src/")
import gwaslab as gl

In [2]:
mysumstats = gl.Sumstats("t2d_bbj.txt.gz",
             snpid="SNP",
             chrom="CHR",
             pos="POS",
             ea="ALT",
             nea="REF",
             neaf="Frq",
             beta="BETA",
             se="SE",
             p="P",
             direction="Dir",
             n="N")

Wed Nov  2 18:14:29 2022 Start to initiate from file :t2d_bbj.txt.gz
Wed Nov  2 18:14:58 2022  -Reading columns          : P,Dir,REF,BETA,POS,N,ALT,SNP,SE,CHR,Frq
Wed Nov  2 18:14:58 2022  -Renaming columns to      : P,DIRECTION,NEA,BETA,POS,N,EA,SNPID,SE,CHR,EAF
Wed Nov  2 18:14:58 2022  -Current dataframe shape  : Rows  12557761  x  11  Columns
Wed Nov  2 18:15:04 2022  -Initiating a status column ...
Wed Nov  2 18:15:07 2022  -NEAF is specified...
Wed Nov  2 18:15:07 2022  -Checking if 0<= NEAF <=1 ...
Wed Nov  2 18:15:11 2022  -Converted NEAF to EAF.
Wed Nov  2 18:15:11 2022  -Removed 0 variants with bad NEAF.
Wed Nov  2 18:15:11 2022  -Reordering columns to    : SNPID,CHR,POS,EA,NEA,EAF,BETA,SE,P,N,DIRECTION,STATUS
Wed Nov  2 18:15:15 2022 Finished loading data successfully!


In [3]:
mysumstats.random_variants(n=100000,inplace=True)

Wed Nov  2 18:15:15 2022 Start to randomly select variants from the sumstats...
Wed Nov  2 18:15:15 2022  -Number of variants selected from the sumstats: 100000
Wed Nov  2 18:15:16 2022 Finished sampling...


## basic_check first

In [4]:
mysumstats.basic_check()

Wed Nov  2 18:15:17 2022 Start to check IDs...
Wed Nov  2 18:15:17 2022  -Current Dataframe shape : 100000  x  12
Wed Nov  2 18:15:17 2022  -Checking if SNPID is chr:pos:ref:alt...(separator: - ,: , _)
Wed Nov  2 18:15:18 2022 Finished checking IDs successfully!
Wed Nov  2 18:15:18 2022 Start to fix chromosome notation...
Wed Nov  2 18:15:18 2022  -Current Dataframe shape : 100000  x  12
Wed Nov  2 18:15:18 2022  -Vairants with fixable chromosome notations: 2578
Wed Nov  2 18:15:18 2022  -Converting to string datatype and UPPERCASE...
Wed Nov  2 18:15:18 2022  -Stripping chr prefix if exists : CHR_-.0...
Wed Nov  2 18:15:18 2022  -Identified  2578  variants on sex chromosomes...
Wed Nov  2 18:15:18 2022  -Standardizing sex chromosome notations: X Y MT  to 23,24,25...
Wed Nov  2 18:15:18 2022  -No unrecognized chromosome notations...
Wed Nov  2 18:15:18 2022 Finished fixing chromosome notation successfully!
Wed Nov  2 18:15:18 2022 Start to fix basepair positions...
Wed Nov  2 18:15:18 

In [5]:
mysumstats.data

,SNPID,CHR,POS,EA,NEA,EAF,BETA,SE,P,N,DIRECTION,STATUS
10054101,15:38812350_C_A,15,38812350,C,A,0.9951,0.0599,0.0911,0.5105,191764,+-++,9960099
12083060,22:19698468_C_T,22,19698468,C,T,0.9757,0.0339,0.0447,0.4475,191764,+-++,9960099
2361791,3:74632401_G_T,3,74632401,G,T,0.9973,0.1033,0.0996,0.2995,191764,+++-,9960099
9529731,13:103561507_G_A,13,103561507,G,A,0.9927,0.0147,0.0608,0.8093,191764,-+-+,9960099
7398350,10:14720492_T_G,10,14720492,G,T,0.0439,-0.0297,0.0220,0.1772,191764,---+,9960099
...,...,...,...,...,...,...,...,...,...,...,...,...
2060928,3:10150852_C_G,3,10150852,G,C,0.0035,-0.1704,0.1423,0.2312,166718,-?-+,9960099
11343005,18:70257299_A_G,18,70257299,G,A,0.9334,0.0066,0.0175,0.7085,191764,++--,9960099
5965701,7:127359900_G_C,7,127359900,G,C,0.9984,-0.1798,0.1669,0.2814,191764,-+--,9960099
5626961,7:44311429_G_A,7,44311429,G,A,0.9499,0.0186,0.0200,0.3530,191764,-+++,9960099


## filter_value : a wrapper of pd.DataFrame query

In [6]:
mysumstats.filter_value('1>BETA>0 & EA=="G"')

Wed Nov  2 18:15:28 2022 Start filtering values by condition: 1>BETA>0 & EA=="G"
Wed Nov  2 18:15:28 2022  -Removing 77036 variants not meeting the conditions: 1>BETA>0 & EA=="G"
Wed Nov  2 18:15:28 2022 Finished filtering values.


,SNPID,CHR,POS,EA,NEA,EAF,BETA,SE,P,N,DIRECTION,STATUS
2361791,3:74632401_G_T,3,74632401,G,T,0.9973,0.1033,0.0996,0.29950,191764,+++-,9960099
9529731,13:103561507_G_A,13,103561507,G,A,0.9927,0.0147,0.0608,0.80930,191764,-+-+,9960099
1824893,2:203820984_T_G,2,203820984,G,T,0.0416,0.0106,0.0277,0.70250,191764,-++-,9960099
6602054,8:107036527_A_G,8,107036527,G,A,0.4291,0.0016,0.0088,0.85760,191764,0++-,9960099
4266931,5:114810433_G_C,5,114810433,G,C,0.9988,0.1704,0.1327,0.19920,191764,+-++,9960099
...,...,...,...,...,...,...,...,...,...,...,...,...
5254533,6:143188247_G_A,6,143188247,G,A,0.6887,0.0189,0.0095,0.04705,191764,-+++,9960099
4559957,5:179991841_A_G,5,179991841,G,A,0.1710,0.0068,0.0121,0.57300,191764,-+-+,9960099
12054691,21:44738836_G_A,21,44738836,G,A,0.9986,0.0073,0.1695,0.96550,166718,+?--,9960099
11343005,18:70257299_A_G,18,70257299,G,A,0.9334,0.0066,0.0175,0.70850,191764,++--,9960099


In [7]:
mysumstats.filter_value('1>BETA>0 & EA=="G"',inplace=True)

Wed Nov  2 18:15:29 2022 Start filtering values by condition: 1>BETA>0 & EA=="G"
Wed Nov  2 18:15:29 2022  -Removing 77036 variants not meeting the conditions: 1>BETA>0 & EA=="G"
Wed Nov  2 18:15:29 2022 Finished filtering values.


## Filter variants in specified regions 

In [8]:
mysumstats.filter_region_in(high_ld=True,inplace=False)

Wed Nov  2 18:15:29 2022 Start to sort the genome coordinates...
Wed Nov  2 18:15:29 2022  -Current Dataframe shape : 22964  x  12
Wed Nov  2 18:15:29 2022  -Force converting POS to integers...
Wed Nov  2 18:15:29 2022  -Sorting genome coordinates...
Wed Nov  2 18:15:29 2022 Finished sorting genome coordinates successfully!
Wed Nov  2 18:15:29 2022 Start to filter in variants if in intervals defined in bed files:
Wed Nov  2 18:15:29 2022  -Current Dataframe shape : 22964  x  12
Wed Nov  2 18:15:29 2022  -Loading bed format file for hg19
Wed Nov  2 18:15:29 2022  -Bed file < 100 lines: using pd IntervalIndex... 
Wed Nov  2 18:15:29 2022  -Number of variants in the specified regions to keep: 815
Wed Nov  2 18:15:29 2022  -Number of variants removed: 0
Wed Nov  2 18:15:29 2022 Finished filtering in variants.


,SNPID,CHR,POS,EA,NEA,EAF,BETA,SE,P,N,DIRECTION,STATUS
379,1:48030457_G_A,1,48030457,G,A,0.9243,0.0279,0.0166,0.09355,191764,+++-,9960099
380,1:48072905_G_A,1,48072905,G,A,0.9977,0.1239,0.1465,0.39770,191764,+++-,9960099
381,1:48107575_G_C,1,48107575,G,C,0.9561,0.0373,0.0247,0.13020,191764,-+++,9960099
382,1:48135301_G_A,1,48135301,G,A,0.9418,0.0160,0.0214,0.45380,191764,+-++,9960099
383,1:48217635_G_A,1,48217635,G,A,0.9975,0.0846,0.1417,0.55040,191764,+-+-,9960099
...,...,...,...,...,...,...,...,...,...,...,...,...
21498,20:33581955_G_A,20,33581955,G,A,0.4030,0.0027,0.0088,0.76180,191764,+-+-,9960099
21499,20:33775200_G_A,20,33775200,G,A,0.9592,0.0324,0.0221,0.14300,191764,-++-,9960099
21500,20:34267386_A_G,20,34267386,G,A,0.1374,0.0238,0.0128,0.06261,191764,++++,9960099
21501,20:34283626_A_G,20,34283626,G,A,0.0211,0.0393,0.0319,0.21860,191764,-+++,9960099


In [9]:
mysumstats.filter_region_out(high_ld=True,inplace=False)

Wed Nov  2 18:15:29 2022 Start to sort the genome coordinates...
Wed Nov  2 18:15:29 2022  -Current Dataframe shape : 22964  x  12
Wed Nov  2 18:15:29 2022  -Force converting POS to integers...
Wed Nov  2 18:15:29 2022  -Sorting genome coordinates...
Wed Nov  2 18:15:29 2022 Finished sorting genome coordinates successfully!
Wed Nov  2 18:15:29 2022 Start to filter out variants if in intervals defined in bed files:
Wed Nov  2 18:15:29 2022  -Current Dataframe shape : 22964  x  12
Wed Nov  2 18:15:29 2022  -Loading bed format file for hg19
Wed Nov  2 18:15:29 2022  -Bed file < 100 lines: using pd IntervalIndex... 
Wed Nov  2 18:15:29 2022  -Number of variants in the specified regions to exclude: 0
Wed Nov  2 18:15:29 2022  -Number of variants left: 22149
Wed Nov  2 18:15:29 2022 Finished filtering out variants.


,SNPID,CHR,POS,EA,NEA,EAF,BETA,SE,P,N,DIRECTION,STATUS
0,1:777745_G_A,1,777745,G,A,0.9978,0.1153,0.1424,0.418400,166718,-?++,9960099
1,1:848456_A_G,1,848456,G,A,0.1501,0.0036,0.0136,0.788900,191764,+++-,9960099
2,1:1032954_A_G,1,1032954,G,A,0.1780,0.0107,0.0118,0.365500,191764,+++-,9960099
3,1:1180779_G_C,1,1180779,G,C,0.4253,0.0129,0.0111,0.242800,191764,+-+-,9960099
4,1:1332112_A_G,1,1332112,G,A,0.0289,0.0787,0.0274,0.004105,191764,++++,9960099
...,...,...,...,...,...,...,...,...,...,...,...,...
22959,X:152672496_G_A,23,152672496,G,A,0.9930,0.1550,0.0651,0.017270,191764,--++,9960099
22960,X:152766239_A_G,23,152766239,G,A,0.8784,0.0444,0.0108,0.000040,191764,+++-,9960099
22961,X:152766713_G_A,23,152766713,G,A,0.9989,0.1945,0.2032,0.338500,166718,+?+-,9960099
22962,X:152880742_A_G,23,152880742,G,A,0.9859,0.0270,0.0375,0.472300,191764,--+-,9960099
